In [2]:
import torch
from tqdm.notebook import tqdm

import pandas as pd
import numpy as np
from transformers import (AutoTokenizer,
                          T5ForConditionalGeneration)

device = torch.device('cuda' if torch.cuda.is_available() else 
                      ('mps' if torch.backends.mps.is_available() else 'cpu'))

### Preparation

In [ ]:
device

In [ ]:
# downloading summary model
summary_model_name = "IlyaGusev/rut5_base_sum_gazeta"
summary_tokenizer = AutoTokenizer.from_pretrained(summary_model_name)
summary_model = T5ForConditionalGeneration.from_pretrained(summary_model_name).half().to(device)

In [6]:
checkpoints = []
error_list = []

In [7]:
# summarize batches of text
def summary_text(df, tokenizer, model, batch_size=94):
    
    # generating batches from df
    batches = [group.to_list() for name, group in df.text.groupby(np.arange(len(df)) // batch_size)]
    result = []
    
    for batch in tqdm(batches, total=len(batches)):
        
        # in case of crashing
        try:
            input_ids = tokenizer(
                batch,
                max_length=600,
                add_special_tokens=True,
                padding="max_length",
                truncation=True,
                return_tensors="pt"
            )["input_ids"].to(device)

            output_ids = model.generate(
                input_ids=input_ids,
                no_repeat_ngram_size=3,
                early_stopping=True,
                num_beams=2
            )
            
            summaries = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
            
            result.extend(summaries)
            
            # saving checkpoints in case of crashing
            # storing checkpoints in the yandex datasphere with %
            checkpoints.extend(summaries)
            %store checkpoints
            
        except Exception as e:
            error_list.append((len(result), len(result)+len(batch)))

    return result

In [8]:
df = pd.read_csv('/home/jupyter/datasphere/project/preprocessed.csv', sep=';')
df = df[['Текст инцидента', 'Группа тем']]
df.columns = ['text', 'label']

### Summarizing and saving texts

In [9]:
# summarizing text 
# it took about 2+ hours
summaries = summary_text(df, summary_tokenizer, summary_model)

  0%|          | 0/243 [00:00<?, ?it/s]

Stored 'checkpoints' (list)
Stored 'checkpoints' (list)
Stored 'checkpoints' (list)
Stored 'checkpoints' (list)
Stored 'checkpoints' (list)
Stored 'checkpoints' (list)
Stored 'checkpoints' (list)
Stored 'checkpoints' (list)
Stored 'checkpoints' (list)
Stored 'checkpoints' (list)
Stored 'checkpoints' (list)
Stored 'checkpoints' (list)
Stored 'checkpoints' (list)
Stored 'checkpoints' (list)
Stored 'checkpoints' (list)
Stored 'checkpoints' (list)
Stored 'checkpoints' (list)
Stored 'checkpoints' (list)
Stored 'checkpoints' (list)
Stored 'checkpoints' (list)
Stored 'checkpoints' (list)
Stored 'checkpoints' (list)
Stored 'checkpoints' (list)
Stored 'checkpoints' (list)
Stored 'checkpoints' (list)
Stored 'checkpoints' (list)
Stored 'checkpoints' (list)
Stored 'checkpoints' (list)
Stored 'checkpoints' (list)
Stored 'checkpoints' (list)
Stored 'checkpoints' (list)
Stored 'checkpoints' (list)
Stored 'checkpoints' (list)
Stored 'checkpoints' (list)
Stored 'checkpoints' (list)
Stored 'checkpoints'

In [11]:
# checking index'es of rows in batches with error
error_list

[(9588, 9682), (11938, 12032)]

In [13]:
len(summaries)

22592

In [18]:
df_for_smol = pd.read_csv('/home/jupyter/datasphere/project/preprocessed.csv', sep=';')

In [19]:
# mapping and cutting labels with summaries 
# considering batches errors
big_labels = pd.concat([df.label.iloc[:9588], 
                        df.label.iloc[9682:11938], 
                        df.label.iloc[12032:]], 
                       ignore_index=True)

smol_labels = pd.concat([df_for_smol["Тема"].iloc[:9588], 
                         df_for_smol["Тема"].iloc[9682:11938], 
                         df_for_smol["Тема"].iloc[12032:]], 
                        ignore_index=True)

In [20]:
big_labels.shape

(22592,)

In [21]:
# checking mapping
big_labels.shape[0] == smol_labels.shape[0] == len(summaries)

True

In [24]:
# saving summaries
df_summaries = pd.DataFrame(data={
    "text": summaries,
    "big_labels": big_labels,
    "smol_labels": smol_labels
})
df_summaries

,text,big_labels,smol_labels
0,"В яму упала мать, упала ребенок. Сильно ударил...",Благоустройство,★ Ямы во дворах
1,В Перми с ноября года не работает социальное т...,Социальное обслуживание и защита,Оказание гос. соц. помощи
2,"Если подавала на пособие с до декабря, когда м...",Социальное обслуживание и защита,Дети и многодетные семьи
3,На остановках можно будет только укрыться от н...,Общественный транспорт,Содержание остановок
4,В Березниках у сына принятого откоронавируса з...,Здравоохранение/Медицина,Технические проблемы с записью на прием к врачу
...,...,...,...
22587,Березники и Березники могут поучаствовать в пр...,Социальное обслуживание и защита,Улучшение жилищных условий
22588,В Гремячинск ситуация с теплой на улице Ленина...,ЖКХ,Ненадлежащее качество или отсутствие отопления
22589,"У меня месячный ребёнок, тест на ковид положит...",Здравоохранение/Медицина,Технические проблемы с записью на прием к врачу
22590,В ближайшее время Федосеев и его коллега по ра...,Благоустройство,Благоустройство придомовых территорий


In [26]:
df["small_labels"] = df_for_smol["Тема"]
df.columns = ['text', 'big_labels', 'smol_labels']
df

,text,big_labels,smol_labels
0,"Добрый день. Сегодня,.., моя мать шла по улице...",Благоустройство,★ Ямы во дворах
1,"Пермь г,. В Перми с ноября года не работает со...",Социальное обслуживание и защита,Оказание гос. соц. помощи
2,Добрый день! Скажите пожалуйста если подавала ...,Социальное обслуживание и защита,Дети и многодетные семьи
3,Каждая из них не о чем. Люди на остановках хот...,Общественный транспорт,Содержание остановок
4,В Березниках у сына принятого откоронавируса з...,Здравоохранение/Медицина,Технические проблемы с записью на прием к врачу
...,...,...,...
22775,"Березники официальные, а если ещё не погасили ...",Социальное обслуживание и защита,Улучшение жилищных условий
22776,Город Гремячинск ситуация с теплом на улице Ле...,ЖКХ,Ненадлежащее качество или отсутствие отопления
22777,"Здравствуйте у меня ребёнку месяца, тест на ко...",Здравоохранение/Медицина,Технические проблемы с записью на прием к врачу
22778,А что творится с благоустройством дворов! Вооб...,Благоустройство,Благоустройство придомовых территорий


In [27]:
df_augment = pd.read_csv("/home/jupyter/datasphere/project/augmented_translator.csv")
df_augment

,text,big_labels,smol_labels
0,Мусоросжигательный завод строится! Кладбище бо...,Погребение и похоронное дело,Погребение и похоронное дело
1,"Хочу добавить к посту, почему больницы не сооб...",Погребение и похоронное дело,Погребение и похоронное дело
2,В нашей Красновишерской скорой помощи при звон...,Погребение и похоронное дело,Погребение и похоронное дело
3,В Кудимкале похоронные компании платят налоги ...,Погребение и похоронное дело,Погребение и похоронное дело
4,"Крематории, безусловно, нужны. Посмотрите коли...",Погребение и похоронное дело,Погребение и похоронное дело
...,...,...,...
997,"город Пермь. Добрый вечер, адрес: ул. Водонагр...",Спецпроекты,Спецпроекты
998,Мир вам. Что значит побег? Почему вам следует ...,Спецпроекты,Спецпроекты
999,город Пермь. Спокойной ночи. Шум в этом году н...,Спецпроекты,Спецпроекты
1000,город Пермь. Куда писать негативные отзывы о б...,Спецпроекты,Спецпроекты


In [28]:
data = pd.concat([df_summaries, df_augment], ignore_index=True)
data.shape

(23594, 3)

In [30]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23594 entries, 0 to 23593
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   text         23594 non-null  object
 1   big_labels   23594 non-null  object
 2   smol_labels  23594 non-null  object
dtypes: object(3)
memory usage: 553.1+ KB


In [31]:
data.to_csv('augmented.csv', index=False)